In [ ]:
import random
import torch
from tqdm import tqdm 



import pickle

class Spelldate(torch.utils.data.Dataset):
    def __init__(self, model):
        self._data = []
        self._label = []
        self.data = None
        self.label = None 

        with open('X.pkl','rb') as f:
            X = pickle.load(f)

        with open('Y.pkl','rb') as f:
            Y = pickle.load(f)
        if model==1:
            for i in tqdm(range(len(X))):
                temp = []
                Tword = create_cyrillic_vector(X[i]['true'])
                for s in range(5):
                    temp.append(Tword - create_cyrillic_vector(X[i]['wlist'][s]))
                self._label.append(1)
                self._data.append(torch.stack(temp))  

            for i in tqdm(range(len(Y))):
                temp = []
                Tword = create_cyrillic_vector(Y[i]['true'])
                for s in range(5):
                    temp.append(Tword - create_cyrillic_vector(Y[i]['wlist'][s]))
                self._label.append(0)
                self._data.append(torch.stack(temp))  
        elif model==2:
            for i in tqdm(range(len(X))):
                temp = []
                for s in range(5):
                    levenshtein_result = levenshtein_distance_and_operations(X[i]['true'],X[i]['wlist'][s])
                    temp.append(torch.tensor(levenshtein_result + (X[i]['sim'][s],)))
                self._label.append(1)
                self._data.append(torch.stack(temp))  

            for i in tqdm(range(len(Y))):
                temp = []
                for s in range(5):
                    levenshtein_result = levenshtein_distance_and_operations(Y[i]['true'],Y[i]['wlist'][s])
                    temp.append(torch.tensor(levenshtein_result + (Y[i]['sim'][s],)))
                self._label.append(0)
                self._data.append(torch.stack(temp))  
                

        # 打乱
        index = [i for i in range(len(self._data))]
        random.shuffle(index)
        self._data = [self._data[i] for i in index]
        self._label = [self._label[i] for i in index]
        
        test_size = len(self._data) // 10
        # 使用切片操作来分割数据和标签
        self.test_data = self._data[:test_size]
        self.test_label = self._label[:test_size]
        self.train_data = self._data[test_size:]
        self.train_label = self._label[test_size:]

    def set_testOrtrain(self, mod):
        if mod == 1:
            self.data = self.train_data
            self.label = self.train_label
        else:
            self.data = self.test_data
            self.label = self.test_label

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.label[idx]
    
spelldate = Spelldate(2)  # 创建数据集
spelldate.set_testOrtrain(1)
from torch.utils.data import DataLoader, TensorDataset
traindataloader = DataLoader(spelldate, batch_size=32)


In [ ]:
import torch.nn as nn
import torch.optim as optim  # 导入optim模块
import torch
import torch.nn as nn
import torch.nn as nn

class spellClass(nn.Module):
    def __init__(self):
        super(spellClass, self).__init__()

        self.fc0 = nn.Linear(5*5, 100)
        self.fc1 = nn.Linear(100, 25)
        self.fc2 = nn.Linear(25, 25)
        self.fc3 = nn.Linear(25, 2)

    def forward(self, x):
        # 将输入数据展平为一维
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc0(x))
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = spellClass()  
criterion = nn.CrossEntropyLoss()  # 定义损失函数为交叉熵损失
optimizer = optim.Adam(net.parameters(), lr=2e-3)  # 定义优化器为Adam，学习率为0.002

In [ ]:


def train():
    net.train()  # 将模型设置为训练模式
    for i, (images, labels) in enumerate(traindataloader):  # 使用枚举函数遍历数据和标签
        optimizer.zero_grad()  # 清零梯度
        output = net(images)  # 前向传播
        loss = criterion(output, labels)  # 计算损失
        # 每10个批次打印一次损失
        if i % 500 == 0:
            print('Train - , Batch: %d, Loss: %f' % ( i, loss.detach().cpu().item()))
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权重

# def test():
#     net.eval()  # 将模型设置为评估模式
#     total_correct = 0
#     avg_loss = 0.0
#     for i, (images, labels) in enumerate(zip(spelldate.test_data, torch.tensor(spelldate.test_label))):
#         output = net(images)  # 前向传播
#         avg_loss += criterion(output, labels).sum()  # 累计损失
#         pred = output.detach().max(1)[1]  # 获取预测结果
#         total_correct += pred.eq(labels.view_as(pred)).sum()  # 计算正确预测的数量

#     avg_loss /= len(spelldate.test_data)  # 计算平均损失
#     print('Test Avg. Loss: %f, Accuracy: %f' % (avg_loss.detach().cpu().item(), float(total_correct) / len(spelldate.test_data)))  # 打印平均损失和准确率
train()  # 训练模型
train()  # 训练模型
train()  # 训练模型
train()  # 训练模型
train()  # 训练模型
train()  # 训练模型
train()  # 训练模型
# test()  # 测试模型


In [ ]:


# 示例使用
text = "привет как дела"
vector = create_cyrillic_vector(text)
print(vector)
len(vector)


In [ ]:

# 示例
s1 = "kitten"
s2 = "sitting"
a = levenshtein_distance_and_operations(s1, s2)
a

In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForMaskedLM


In [ ]:

tokenizer = AutoTokenizer.from_pretrained("hfl/cino-large-v2")
model = AutoModelForMaskedLM.from_pretrained("hfl/cino-large-v2")


tokenizer = AutoTokenizer.from_pretrained("tugstugi/bert-base-mongolian-cased")
model = AutoModelForMaskedLM.from_pretrained("tugstugi/bert-base-mongolian-cased")


tokenizer = AutoTokenizer.from_pretrained("tugstugi/bert-base-mongolian-uncased")
model = AutoModelForMaskedLM.from_pretrained("tugstugi/bert-base-mongolian-uncased")


tokenizer = AutoTokenizer.from_pretrained("tugstugi/bert-large-mongolian-cased")
model = AutoModelForMaskedLM.from_pretrained("tugstugi/bert-large-mongolian-cased")


tokenizer = AutoTokenizer.from_pretrained("tugstugi/bert-large-mongolian-uncased")
model = AutoModelForMaskedLM.from_pretrained("tugstugi/bert-large-mongolian-uncased")


tokenizer = AutoTokenizer.from_pretrained("phjhk/hklegal-xlm-r-base")
model = AutoModelForMaskedLM.from_pretrained("phjhk/hklegal-xlm-r-base")


tokenizer = AutoTokenizer.from_pretrained("Twitter/twhin-bert-large")
model = AutoModelForMaskedLM.from_pretrained("Twitter/twhin-bert-large")



tokenizer = AutoTokenizer.from_pretrained("3ebdola/Dialectal-Arabic-XLM-R-Base")
model = AutoModelForMaskedLM.from_pretrained("3ebdola/Dialectal-Arabic-XLM-R-Base")


tokenizer = AutoTokenizer.from_pretrained("phjhk/hklegal-xlm-r-large")
model = AutoModelForMaskedLM.from_pretrained("phjhk/hklegal-xlm-r-large")


tokenizer = AutoTokenizer.from_pretrained("phjhk/hklegal-xlm-r-large-t")
model = AutoModelForMaskedLM.from_pretrained("phjhk/hklegal-xlm-r-large-t")


tokenizer = AutoTokenizer.from_pretrained("plAIground/xlmr-bert-multilingual-base-merge")
model = AutoModelForMaskedLM.from_pretrained("plAIground/xlmr-bert-multilingual-base-merge")

In [ ]:

def cross(Alist, Blist):
    return [a + b for a in Alist for b in Blist]


cross(['a','b','c'], ['1','2','3'])

In [ ]:
a = [1,2,3,4]
a[:2]

In [ ]:
a * 2

In [ ]:
import random

def generate_spelling_error(word):
    # 蒙古语西里尔字母表
    cyrillic_letters = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

    # 确保输入是小写字母
    word = word.lower()

    # 随机选择错误类型
    error_type = random.choice(['missing', 'extra', 'swap'])

    if error_type == 'missing' and len(word) > 1:
        # 缺失一个字符
        index = random.randint(0, len(word) - 1)
        error_word = word[:index] + word[index + 1:]
    elif error_type == 'extra':
        # 多出一个字符
        index = random.randint(0, len(word))
        extra_char = random.choice(cyrillic_letters)
        error_word = word[:index] + extra_char + word[index:]
    elif error_type == 'swap' and len(word) > 1:
        # 交换两个字符
        index1 = random.randint(0, len(word) - 2)
        index2 = index1 + 1
        error_word = word[:index1] + word[index2] + word[index1] + word[index2 + 1:]
    else:
        # 如果单词长度为1且选择了缺失或交换，则返回原单词
        error_word = word

    return error_word

# 示例用法
word = 'монгол'
error_word = generate_spelling_error(word)
print("原单词:", word)
print("拼写错误单词:", error_word)
